# Day 2: Rock Paper Scissors

## Part 1 

The Elves begin to set up camp on the beach. To decide whose tent gets to be closest to the snack storage, a giant Rock Paper Scissors tournament is already in progress.

Rock Paper Scissors is a game between two players. Each game contains many rounds; in each round, the players each simultaneously choose one of Rock, Paper, or Scissors using a hand shape. Then, a winner for that round is selected: Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. If both players choose the same shape, the round instead ends in a draw.

Appreciative of your help yesterday, one Elf gives you an encrypted strategy guide (your puzzle input) that they say will be sure to help you win. "The first column is what your opponent is going to play: A for Rock, B for Paper, and C for Scissors. The second column--" Suddenly, the Elf is called away to help with someone's tent.

The second column, you reason, must be what you should play in response: X for Rock, Y for Paper, and Z for Scissors. Winning every time would be suspicious, so the responses must have been carefully chosen.

The winner of the whole tournament is the player with the highest score. Your total score is the sum of your scores for each round. The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).

Since you can't be sure if the Elf is trying to help you or trick you, you should calculate the score you would get if you were to follow the strategy guide.

For example, suppose you were given the following strategy guide:
```
A Y
B X
C Z
```
This strategy guide predicts and recommends the following:

In the first round, your opponent will choose Rock (A), and you should choose Paper (Y). This ends in a win for you with a score of 8 (2 because you chose Paper + 6 because you won).
In the second round, your opponent will choose Paper (B), and you should choose Rock (X). This ends in a loss for you with a score of 1 (1 + 0).
The third round is a draw with both players choosing Scissors, giving you a score of 3 + 3 = 6.
In this example, if you were to follow the strategy guide, you would get a total score of 15 (8 + 1 + 6).

What would your total score be if everything goes exactly according to your strategy guide?

In [1]:
%load_ext nb_mypy

Version 1.0.4


In [2]:
from __future__ import annotations

from typing import ClassVar


class Item:
    score: ClassVar[int]

    @property
    def beats(self) -> type[Item]:
        # can't define this as a class level attribute because classes hasn't been defined yet
        raise NotImplementedError(
            f"{self.__class__.__name__} must return the class it beats"
        )

    def __gt__(self, other: Item) -> bool:
        return isinstance(other, self.beats)

    def __eq__(self, other: object) -> bool:
        return self.__class__ == other.__class__

    def __repr__(self) -> str:
        return self.__class__.__name__ + "!"

    def __hash__(self) -> int:
        """Return ASCII code of the first character."""
        # Needed for lru & set operations in part 2
        return ord(self.__class__.__name__[0])


class Rock(Item):
    score = 1

    @property
    def beats(self):
        return Scissors


class Paper(Item):
    score = 2

    @property
    def beats(self):
        return Rock


class Scissors(Item):
    score = 3

    @property
    def beats(self):
        return Paper


r = Rock()
p = Paper()
s = Scissors()

assert r == Rock()
assert r > s
assert s < r
assert s > p
assert p > r

r > s > p > r

True

In [3]:
import enum


class OpponentPick(str, enum.Enum):
    ROCK = "A"
    PAPER = "B"
    SCISSORS = "C"


class MyPick(str, enum.Enum):
    ROCK = "X"
    PAPER = "Y"
    SCISSORS = "Z"


def rps_factory(char: OpponentPick | MyPick | str) -> Rock | Paper | Scissors:
    # TODO: pattern matching
    class_mapping: dict[str, type[Rock] | type[Paper] | type[Scissors]] = {
        OpponentPick.ROCK: Rock,
        MyPick.ROCK: Rock,
        OpponentPick.PAPER: Paper,
        MyPick.PAPER: Paper,
        OpponentPick.SCISSORS: Scissors,
        MyPick.SCISSORS: Scissors,
    }
    item_class = class_mapping[char]
    return item_class()


rps_factory("A") == rps_factory(OpponentPick.ROCK) == rps_factory(MyPick.ROCK)

True

In [4]:
from pathlib import Path
from typing import Generator

input_file = Path.cwd().parent / "inputs" / "day2.txt"
assert input_file.exists(), input_file


def ingest_rps_picks(input_file: Path) -> Generator[tuple[Item, Item], None, None]:
    with open(input_file) as f_in:
        for line in f_in:
            opponent_pick, my_pick = line.split()
            yield rps_factory(opponent_pick), rps_factory(my_pick)


g = ingest_rps_picks(input_file)
next(g)

(Scissors!, Rock!)

In [5]:
WIN: int = 6
DRAW: int = 3
LOSS: int = 0

my_score: int = 0
opp_score: int = 0

for opp_pick, my_pick in ingest_rps_picks(input_file):
    my_score += my_pick.score
    opp_score += opp_pick.score

    if my_pick == opp_pick:
        my_score += DRAW
        opp_score += DRAW
    elif my_pick > opp_pick:
        my_score += WIN
        opp_score += LOSS
    elif opp_pick > my_pick:
        my_score += LOSS
        opp_score += WIN
    else:
        raise NotImplementedError("Unknown Condition Encountered")

print(f"{my_score=}\n{opp_score=}")

my_score=13221
opp_score=12734


## Part 2

The Elf finishes helping with the tent and sneaks back over to you. "Anyway, the second column says how the round needs to end: X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win. Good luck!"

The total score is still calculated in the same way, but now you need to figure out what shape to choose so the round ends as indicated. The example above now goes like this:

In the first round, your opponent will choose Rock (A), and you need the round to end in a draw (Y), so you also choose Rock. This gives you a score of 1 + 3 = 4.
In the second round, your opponent will choose Paper (B), and you choose Rock so you lose (X) with a score of 1 + 0 = 1.
In the third round, you will defeat your opponent's Scissors with Rock for a score of 1 + 6 = 7.
Now that you're correctly decrypting the ultra top secret strategy guide, you would get a total score of 12.

Following the Elf's instructions for the second column, **what would your total score be if everything goes exactly according to your strategy guide?**

In [6]:
import enum
import functools
from typing import TypeAlias


class OpponentPick(str, enum.Enum):
    ROCK = "A"
    PAPER = "B"
    SCISSORS = "C"


class MyStrategy(str, enum.Enum):
    LOSE = "X"
    DRAW = "Y"
    WIN = "Z"


RPS: TypeAlias = Rock | Paper | Scissors


@functools.lru_cache(maxsize=32)
def _select_rps_for_strategy(opp_rps: RPS, my_strat: MyStrategy) -> RPS:
    options: set[RPS] = {Rock(), Paper(), Scissors()}

    if my_strat == MyStrategy.DRAW:
        return opp_rps
    options.remove(opp_rps)  # only needed for draw

    for choice in options:
        if my_strat == MyStrategy.WIN and choice > opp_rps:
            return choice
        if my_strat == MyStrategy.LOSE and choice < opp_rps:
            return choice

    raise NotImplementedError("Don't know how to satisfy strategy")


def rps_factory_2(opp_pick: OpponentPick, my_strat: MyStrategy) -> tuple[RPS, RPS]:
    pick_mapping: dict[str, RPS] = {
        OpponentPick.ROCK: Rock(),
        OpponentPick.PAPER: Paper(),
        OpponentPick.SCISSORS: Scissors(),
    }

    opp_rps_pick: RPS = pick_mapping[opp_pick]

    my_rps_pick: RPS = _select_rps_for_strategy(opp_rps_pick, my_strat)

    return opp_rps_pick, my_rps_pick


print(rps_factory_2(OpponentPick.PAPER, MyStrategy.WIN))
print(rps_factory_2(OpponentPick.PAPER, MyStrategy.LOSE))
print(rps_factory_2(OpponentPick.PAPER, MyStrategy.DRAW))

(Paper!, Scissors!)
(Paper!, Rock!)
(Paper!, Paper!)


In [7]:
def ingest_rps_picks_2(input_file: Path) -> Generator[tuple[Item, Item], None, None]:
    with open(input_file) as f_in:
        for line in f_in:
            opponent_pick, my_strat = line.split()
            yield rps_factory_2(opponent_pick, my_strat)  # type: ignore[arg-type]


g = ingest_rps_picks_2(input_file)
next(g)

(Scissors!, Paper!)

In [8]:
WIN: int = 6
DRAW: int = 3
LOSS: int = 0

my_score: int = 0
opp_score: int = 0

for opp_pick, my_pick in ingest_rps_picks_2(input_file):
    my_score += my_pick.score
    opp_score += opp_pick.score

    if my_pick == opp_pick:
        my_score += DRAW
        opp_score += DRAW
    elif my_pick > opp_pick:
        my_score += WIN
        opp_score += LOSS
    elif opp_pick > my_pick:
        my_score += LOSS
        opp_score += WIN
    else:
        raise NotImplementedError("Unknown Condition Encountered")

print(f"{my_score=}\n{opp_score=}")

my_score=13131
opp_score=12239
